In [1]:
#model parameter 확인
for name, param in model.named_parameters(): 
    print(f'name:{name}') 
    print(type(param)) 
    print(f'param.shape:{param.shape}') 
    print(f'param.requries_grad:{param.requires_grad}') 
    print('=====')

NameError: name 'model' is not defined

In [10]:
import torch.nn as nn

In [11]:
model.patch_embed.proj=nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))

In [1]:
import argparse
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import copy
import random
from tqdm import tqdm
#sys.path.append("..")
from tools import utils
#from model.video_kwsnet import KWS_Net
from reader.data_reader_video import myDataLoader, myDataset


In [2]:
def seed_torch(seed=42):
    seed = int(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True


In [3]:
model_path = 'video_model'
log_dir = './log/'
logger = utils.get_logger(log_dir + '/' + 'video_model')
seed_torch(617)

# load mean and var
lip_train_mean_var = np.load('scp_dir/train_mean_var_lip.npz')
lip_train_mean = lip_train_mean_var['_mean']
lip_train_var = lip_train_mean_var['_var']

# file path
file_train_positive_path  = '/home/hj20/misp2021_baseline/task1_wws/kws_net_only_video/scp_dir/positive_train.scp'
file_train_negative_path  = '/home/hj20/misp2021_baseline/task1_wws/kws_net_only_video/scp_dir/negative_train.scp'
file_dev_positive_path  = '/home/hj20/misp2021_baseline/task1_wws/kws_net_only_video/scp_dir/positive_dev.scp'
file_dev_negative_path = '/home/hj20/misp2021_baseline/task1_wws/kws_net_only_video/scp_dir/negative_dev.scp'
    
# define the dataloader
print("loading the dataset ...")
dataset_train = myDataset(file_train_positive_path, file_train_negative_path, lip_train_mean, lip_train_var)
dataset_dev = myDataset(file_dev_positive_path, file_dev_negative_path, lip_train_mean, lip_train_var)
dataloader_train = myDataLoader(dataset=dataset_train,
                            batch_size=4,
                            shuffle=True,
                            num_workers=6)
dataloader_dev = myDataLoader(dataset=dataset_dev,
                            batch_size=4,
                            shuffle=False,
                            num_workers=1)
print("- done.")
all_file = len(dataloader_train)
all_file_dev = len(dataloader_dev)
print("- {} training samples, {} dev samples".format(len(dataset_train), len(dataset_dev)))
print("- {} training batch, {} dev batch".format(len(dataloader_train), len(dataloader_dev)))
    

loading the dataset ...
- done.
- 51269 training samples, 2705 dev samples
- 12818 training batch, 677 dev batch


In [62]:
x= x.torch

AttributeError: 'Tensor' object has no attribute 'torch'

In [75]:
x=dataset_train[3][0]

In [76]:
x = torch.from_numpy(x).cuda()

In [77]:
x = x.unsqueeze(0)

In [78]:
x.shape

torch.Size([1, 33, 96, 96])

In [86]:
video_frontend = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False),
            nn.BatchNorm3d(16), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1)))

In [80]:
B, T, _, _ = x.size()

In [81]:
x = x.unsqueeze(1)

In [82]:
x.shape

torch.Size([1, 1, 33, 96, 96])

In [83]:
video_frontend=video_frontend.cuda()

In [84]:
x = video_frontend(x)

In [85]:
x.shape

torch.Size([1, 3, 33, 24, 24])

In [29]:
x = x.transpose(1, 2).contiguous()

In [30]:
x.shape

torch.Size([1, 34, 64, 24, 24])

In [33]:
x

tensor([[[[[6.7833e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 6.1398e-01],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 6.1398e-01],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 5.6885e-01],
           ...,
           [0.0000e+00, 1.2454e-01, 1.0277e-01,  ..., 2.0375e+00,
            2.3183e+00, 2.4504e+00],
           [3.0174e-01, 6.9994e-01, 7.7454e-01,  ..., 2.1981e+00,
            2.3183e+00, 2.4504e+00],
           [5.4947e-01, 5.9432e-01, 5.9432e-01,  ..., 2.5050e+00,
            2.5098e+00, 2.5456e+00]],

          [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [4.6002e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [1.4307e-01, 3.8793e-02, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [1.4681e+00, 1.4681e+00, 1.4

In [41]:
model = PaSST().cuda()

In [37]:
x.shape

torch.Size([1, 22, 64, 24, 24])

In [44]:
model(x)[0].shape

torch.Size([22, 1])

In [31]:
x = x.view(-1, 64, x.size(3), x.size(4))

In [32]:
x.shape

torch.Size([34, 64, 24, 24])

In [40]:
resnet

<bound method Module.cuda of ResNet(
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 

In [41]:
resnet(x)

TypeError: cuda(): argument 'device' (position 1) must be torch.device, not Tensor

In [4]:
#passt

In [5]:
import math
import logging
import warnings
from functools import partial
from collections import OrderedDict
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F

from timm.models.layers.helpers import to_2tuple

#from .helpers.vit_helpers import update_default_cfg_and_kwargs, DropPath, trunc_normal_, build_model_with_cfg

In [6]:
def adapt_input_conv(in_chans, conv_weight):
    conv_type = conv_weight.dtype
    conv_weight = conv_weight.float()  # Some weights are in torch.half, ensure it's float for sum on CPU
    O, I, J, K = conv_weight.shape
    if in_chans == 1:
        if I > 3:
            assert conv_weight.shape[1] % 3 == 0
            # For models with space2depth stems
            conv_weight = conv_weight.reshape(O, I // 3, 3, J, K)
            conv_weight = conv_weight.sum(dim=2, keepdim=False)
        else:
            conv_weight = conv_weight.sum(dim=1, keepdim=True)
    elif in_chans != 3:
        if I != 3:
            raise NotImplementedError('Weight format not supported by conversion.')
        else:
            # NOTE this strategy should be better than random init, but there could be other combinations of
            # the original RGB input layer weights that'd work better for specific cases.
            repeat = int(math.ceil(in_chans / 3))
            conv_weight = conv_weight.repeat(1, repeat, 1, 1)[:, :in_chans, :, :]
            conv_weight *= (3 / float(in_chans))
    conv_weight = conv_weight.to(conv_type)
    return conv_weight


class Mlp(nn.Module):
    """ MLP as used in Vision Transformer, MLP-Mixer and related networks
    """

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


first_RUN = True


class PatchEmbed(nn.Module):
    """ 2D Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=16, stride=16, in_chans=3, embed_dim=768, norm_layer=None,
                 flatten=True):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        stride = to_2tuple(stride)
        self.img_size = img_size
        self.patch_size = patch_size
        self.stride = stride
        self.grid_size = (img_size[0] // stride[0], img_size[1] // stride[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.flatten = flatten
        self.embed_dim = embed_dim
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, C, H, W = x.shape
        if not (H == self.img_size[0] and W == self.img_size[1]):
            warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
        # to do maybe replace weights
        x = self.proj(x)
        if self.flatten:
            x = x.flatten(2).transpose(1, 2)  # BCHW -> BNC
        x = self.norm(x)
        if first_RUN: print("self.norm(x)", x.size())
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class PaSST(nn.Module):
    """
    Based on the implementation of Vision Transformer in timm library.
     Take a look at the get_model function, adapting the weights of pretrained imagenet models.
    """

    def __init__(self, u_patchout=0, s_patchout_t=0, s_patchout_f=0, img_size=(24, 24), patch_size=4, stride=4,
                 in_chans=3, num_classes=256, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=True, representation_size=None, distilled=False,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., embed_layer=PatchEmbed, norm_layer=None,
                 act_layer=None, weight_init=''):
        """
        Args:
            u_patchout: Unstructured Patchout integer, number of items to be removed from the final sequence
            s_patchout_t: structured Patchout time integer, number of columns to be removed from the patches grid
            s_patchout_f: structured Patchout Frequency integer, number of rows to be removed from the patches grid
            img_size (int, tuple): input image size
            patch_size (int, tuple): patch size
            in_chans (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (int): embedding dimension
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            distilled (bool): model includes a distillation token and head as in DeiT models
            drop_rate (float): dropout rate
            attn_drop_rate (float): attention dropout rate
            drop_path_rate (float): stochastic depth rate
            embed_layer (nn.Module): patch embedding layer
            norm_layer: (nn.Module): normalization layer
            weight_init: (str): weight init scheme
        """
        super().__init__()
        self.num_classes = num_classes
        self.u_patchout = u_patchout
        self.s_patchout_t = s_patchout_t
        self.s_patchout_f = s_patchout_f
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.num_tokens = 2 if distilled else 1
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU

        self.patch_embed = embed_layer(
            img_size=img_size, patch_size=patch_size, stride=stride, in_chans=in_chans, embed_dim=embed_dim,
            flatten=False)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.dist_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None
        # PaSST
        # refer to https://arxiv.org/abs/2110.05069 Section 2
        self.new_pos_embed = nn.Parameter(torch.zeros(1, self.num_tokens, embed_dim))  # for C and D tokens
        self.freq_new_pos_embed = nn.Parameter(torch.zeros(1, embed_dim, self.patch_embed.grid_size[0], 1))  # | f
        self.time_new_pos_embed = nn.Parameter(torch.zeros(1, embed_dim, 1, self.patch_embed.grid_size[1]))  # __ t
        ####
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks = nn.Sequential(*[
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, drop=drop_rate,
                attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Representation layer
        if representation_size and not distilled:
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ('fc', nn.Linear(embed_dim, representation_size)),
                ('act', nn.Tanh())
            ]))
        else:
            self.pre_logits = nn.Identity()

        # Classifier head(s)
        self.head = nn.Sequential(nn.LayerNorm(self.num_features),
                                  nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity())
        self.head_dist = None
        if distilled:
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()

        self.init_weights(weight_init)

    def init_weights(self, mode=''):
        assert mode in ('jax', 'jax_nlhb', 'nlhb', '')
        head_bias = -math.log(self.num_classes) if 'nlhb' in mode else 0.
        trunc_normal_(self.new_pos_embed, std=.02)
        trunc_normal_(self.freq_new_pos_embed, std=.02)
        trunc_normal_(self.time_new_pos_embed, std=.02)
        if self.dist_token is not None:
            trunc_normal_(self.dist_token, std=.02)
        if mode.startswith('jax'):
            # leave cls token as zeros to match jax impl
            raise RuntimeError("Not supported yet")
        else:
            trunc_normal_(self.cls_token, std=.02)
            self.apply(_init_vit_weights)

    def _init_weights(self, m):
        # this fn left here for compat with downstream users
        _init_vit_weights(m)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'new_pos_embed', 'freq_new_pos_embed', 'time_new_pos_embed', 'cls_token', 'dist_token'}

    def get_classifier(self):
        if self.dist_token is None:
            return self.head
        else:
            return self.head, self.head_dist

    def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()
        if self.num_tokens == 2:
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        global first_RUN  # not jit friendly? use trace instead
        x = self.patch_embed(x)  # [b, e, f, t]
        B_dim, E_dim, F_dim, T_dim = x.shape  # slow
        if first_RUN: print(" patch_embed : ", x.shape)
        # Adding Time/Freq information
        if first_RUN: print(" self.time_new_pos_embed.shape", self.time_new_pos_embed.shape)
        time_new_pos_embed = self.time_new_pos_embed
        if x.shape[-1] != time_new_pos_embed.shape[-1]:
            time_new_pos_embed = time_new_pos_embed[:, :, :, :x.shape[-1]]
            if first_RUN: print(" CUT time_new_pos_embed.shape", time_new_pos_embed.shape)
        x = x + time_new_pos_embed
        if first_RUN: print(" self.freq_new_pos_embed.shape", self.freq_new_pos_embed.shape)
        x = x + self.freq_new_pos_embed

        # Structured Patchout https://arxiv.org/abs/2110.05069 Section 2.2
        if self.training and self.s_patchout_t:
            if first_RUN: print(f"X Before time Patchout of {self.s_patchout_t} ", x.size())
            # ([1, 768, 1, 82])
            random_indices = torch.randperm(T_dim)[:T_dim - self.s_patchout_t].sort().values
            x = x[:, :, :, random_indices]
            if first_RUN: print("X after time Patchout", x.size())
        if self.training and self.s_patchout_f:
            if first_RUN: print(f"X Before Freq Patchout of {self.s_patchout_f} ", x.size())
            # [1, 768, 12, 1]
            random_indices = torch.randperm(F_dim)[:F_dim - self.s_patchout_f].sort().values
            x = x[:, :, random_indices, :]
            if first_RUN: print(" \n X after freq Patchout: ", x.size())
        ###
        # Flatten the sequence
        x = x.flatten(2).transpose(1, 2)
        # Unstructured Patchout
        if first_RUN: print("X flattened", x.size())
        if self.training and self.u_patchout:
            seq_len = x.shape[1]
            random_indices = torch.randperm(seq_len)[:seq_len - self.u_patchout].sort().values
            x = x[:, random_indices, :]
            if first_RUN: print("X After Unstructured Patchout", x.size())
        ####
        # Add the C/D tokens
        if first_RUN: print(" self.new_pos_embed.shape", self.new_pos_embed.shape)
        cls_tokens = self.cls_token.expand(B_dim, -1, -1) + self.new_pos_embed[:, :1, :]
        if first_RUN: print(" self.cls_tokens.shape", cls_tokens.shape)
        if self.dist_token is None:
            x = torch.cat((cls_tokens, x), dim=1)
        else:
            dist_token = self.dist_token.expand(B_dim, -1, -1) + self.new_pos_embed[:, 1:, :]
            if first_RUN: print(" self.dist_token.shape", dist_token.shape)
            x = torch.cat((cls_tokens, dist_token, x), dim=1)

        if first_RUN: print(" final sequence x", x.shape)
        x = self.pos_drop(x)
        x = self.blocks(x)
        if first_RUN: print(f" after {len(self.blocks)} atten blocks x", x.shape)
        x = self.norm(x)
        if self.dist_token is None:
            return self.pre_logits(x[:, 0])
        else:
            return x[:, 0], x[:, 1]

    def forward(self, x):
        global first_RUN
        if first_RUN: print("x", x.size())

        x = self.forward_features(x)

        if self.head_dist is not None:
            features = (x[0] + x[1]) / 2
            if first_RUN: print("forward_features", features.size())
            x = self.head(features)
            if first_RUN: print("head", x.size())
            first_RUN = False
            return x, features
        else:
            features = x
            if first_RUN: print("forward_features", features.size())
            x = self.head(x)
        if first_RUN: print("head", x.size())
        first_RUN = False
        return x, features


def _init_vit_weights(module: nn.Module, name: str = '', head_bias: float = 0., jax_impl: bool = False):
    """ ViT weight initialization
    * When called without n, head_bias, jax_impl args it will behave exactly the same
      as my original init for compatibility with prev hparam / downstream use cases (ie DeiT).
    * When called w/ valid n (module name) and jax_impl=True, will (hopefully) match JAX impl
    """
    if isinstance(module, nn.Linear):
        if name.startswith('head'):
            nn.init.zeros_(module.weight)
            nn.init.constant_(module.bias, head_bias)
        elif name.startswith('pre_logits'):
            lecun_normal_(module.weight)
            nn.init.zeros_(module.bias)
        else:
            if jax_impl:
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    if 'mlp' in name:
                        nn.init.normal_(module.bias, std=1e-6)
                    else:
                        nn.init.zeros_(module.bias)
            else:
                trunc_normal_(module.weight, std=.02)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
    elif jax_impl and isinstance(module, nn.Conv2d):
        # NOTE conv was left to pytorch default in my original init
        lecun_normal_(module.weight)
        if module.bias is not None:
            nn.init.zeros_(module.bias)
    elif isinstance(module, (nn.LayerNorm, nn.GroupNorm, nn.BatchNorm2d)):
        nn.init.zeros_(module.bias)
        nn.init.ones_(module.weight)


def resize_pos_embed(posemb, posemb_new, num_tokens=1, gs_new=(), mode='bicubic'):
    # Rescale the grid of position embeddings when loading from state_dict. Adapted from
    # https://github.com/google-research/vision_transformer/blob/00883dd691c63a6830751563748663526e811cee/vit_jax/checkpoint.py#L224
    _logger.info('Resized position embedding: %s to %s with %s cls/dis tokens', posemb.shape, posemb_new.shape,
                 num_tokens)
    ntok_new = posemb_new.shape[1]
    if num_tokens:
        posemb_tok, posemb_grid = posemb[:, :num_tokens], posemb[0, num_tokens:]
        ntok_new -= num_tokens
    else:
        posemb_tok, posemb_grid = posemb[:, :0], posemb[0]
    gs_old = int(math.sqrt(len(posemb_grid)))
    if not len(gs_new):  # backwards compatibility
        gs_new = [int(math.sqrt(ntok_new))] * 2
    assert len(gs_new) >= 2
    _logger.info('Position embedding grid-size from %s to %s', [gs_old, gs_old], gs_new)
    posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1).permute(0, 3, 1, 2)
    posemb_grid = F.interpolate(posemb_grid, size=gs_new, mode=mode, align_corners=False)
    posemb_grid = posemb_grid.permute(0, 2, 3, 1).reshape(1, gs_new[0] * gs_new[1], -1)
    posemb = torch.cat([posemb_tok, posemb_grid], dim=1)
    return posemb


def adapt_image_pos_embed_to_passt(posemb, num_tokens=1, gs_new=(), mode='bicubic'):
    # Rescale the grid of position embeddings when loading from state_dict. Adapted from
    # https://github.com/google-research/vision_transformer/blob/00883dd691c63a6830751563748663526e811cee/vit_jax/checkpoint.py#L224
    _logger.info('Resized position embedding: %s to %s with %s cls/dis tokens', posemb.shape, gs_new,
                 num_tokens)
    if num_tokens:
        posemb_tok, posemb_grid = posemb[:, :num_tokens], posemb[0, num_tokens:]
    else:
        posemb_tok, posemb_grid = posemb[:, :0], posemb[0]
    gs_old = int(math.sqrt(len(posemb_grid)))

    assert len(gs_new) >= 2
    _logger.info('Position embedding grid-size from %s to %s', [gs_old, gs_old], gs_new)
    posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1).permute(0, 3, 1, 2)
    posemb_grid = F.interpolate(posemb_grid, size=gs_new, mode=mode, align_corners=False)
    freq_new_pos_embed = posemb_grid.mean(dim=3, keepdim=True)
    time_new_pos_embed = posemb_grid.mean(dim=2, keepdim=True)
    _logger.info('New Position cls/dstl embedding %s', posemb_tok.shape)
    _logger.info('New FREQ Position embedding %s', freq_new_pos_embed.shape)
    _logger.info('New TIME Position embedding %s', time_new_pos_embed.shape)
    return posemb_tok, freq_new_pos_embed, time_new_pos_embed


def checkpoint_filter_fn(state_dict, model):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    if 'model' in state_dict:
        # For deit models
        state_dict = state_dict['model']
    state_dict = {k: v for k, v in state_dict.items()}
    if "time_new_pos_embed" not in state_dict:
        # we are working with ImageNet model
        _logger.info("Adapting pos embedding from ImageNet pretrained model to PaSST.")
        v = state_dict.pop("pos_embed")
        new_pos_embed, freq_new_pos_embed, time_new_pos_embed = adapt_image_pos_embed_to_passt(
            v, getattr(model, 'num_tokens', 1), model.patch_embed.grid_size)
        state_dict["new_pos_embed"] = new_pos_embed
        state_dict["freq_new_pos_embed"] = freq_new_pos_embed
        state_dict["time_new_pos_embed"] = time_new_pos_embed

    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k and len(v.shape) < 4:
            # For old models that I trained prior to conv based patchification
            O, I, H, W = model.patch_embed.proj.weight.shape
            v = v.reshape(O, -1, H, W)
        elif k == 'pos_embed' and v.shape != model.pos_embed.shape:
            # this should never occur
            v = resize_pos_embed(
                v, model.pos_embed, getattr(model, 'num_tokens', 1), model.patch_embed.grid_size)
        out_dict[k] = v
    return out_dict


In [7]:
"""
Adapted from https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
"""
import math
import warnings
from copy import deepcopy

import torch
from timm.models.helpers import load_pretrained
from torch import nn


def overlay_external_default_cfg(default_cfg, kwargs):
    """ Overlay 'external_default_cfg' in kwargs on top of default_cfg arg.
    """
    external_default_cfg = kwargs.pop('external_default_cfg', None)
    if external_default_cfg:
        default_cfg.pop('url', None)  # url should come from external cfg
        default_cfg.pop('hf_hub', None)  # hf hub id should come from external cfg
        default_cfg.update(external_default_cfg)


def filter_kwargs(kwargs, names):
    if not kwargs or not names:
        return
    for n in names:
        kwargs.pop(n, None)


def set_default_kwargs(kwargs, names, default_cfg):
    for n in names:
        # for legacy reasons, model __init__args uses img_size + in_chans as separate args while
        # default_cfg has one input_size=(C, H ,W) entry
        if n == 'img_size':
            input_size = default_cfg.get('input_size', None)
            if input_size is not None:
                assert len(input_size) == 3
                kwargs.setdefault(n, input_size[-2:])
        elif n == 'in_chans':
            input_size = default_cfg.get('input_size', None)
            if input_size is not None:
                assert len(input_size) == 3
                kwargs.setdefault(n, input_size[0])
        else:
            default_val = default_cfg.get(n, None)
            if default_val is not None:
                kwargs.setdefault(n, default_cfg[n])


def update_default_cfg_and_kwargs(default_cfg, kwargs, kwargs_filter):
    """ Update the default_cfg and kwargs before passing to model
    FIXME this sequence of overlay default_cfg, set default kwargs, filter kwargs
    could/should be replaced by an improved configuration mechanism
    Args:
        default_cfg: input default_cfg (updated in-place)
        kwargs: keyword args passed to model build fn (updated in-place)
        kwargs_filter: keyword arg keys that must be removed before model __init__
    """
    # Overlay default cfg values from `external_default_cfg` if it exists in kwargs
    overlay_external_default_cfg(default_cfg, kwargs)
    # Set model __init__ args that can be determined by default_cfg (if not already passed as kwargs)
    default_kwarg_names = ('num_classes', 'global_pool', 'in_chans')
    if default_cfg.get('fixed_input_size', False):
        # if fixed_input_size exists and is True, model takes an img_size arg that fixes its input size
        default_kwarg_names += ('img_size',)
    set_default_kwargs(kwargs, names=default_kwarg_names, default_cfg=default_cfg)
    # Filter keyword args for task specific model variants (some 'features only' models, etc.)
    filter_kwargs(kwargs, names=kwargs_filter)



def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)



from torch.nn.init import _calculate_fan_in_and_fan_out


def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)


def variance_scaling_(tensor, scale=1.0, mode='fan_in', distribution='normal'):
    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    if mode == 'fan_in':
        denom = fan_in
    elif mode == 'fan_out':
        denom = fan_out
    elif mode == 'fan_avg':
        denom = (fan_in + fan_out) / 2

    variance = scale / denom

    if distribution == "truncated_normal":
        # constant is stddev of standard normal truncated to (-2, 2)
        trunc_normal_(tensor, std=math.sqrt(variance) / .87962566103423978)
    elif distribution == "normal":
        tensor.normal_(std=math.sqrt(variance))
    elif distribution == "uniform":
        bound = math.sqrt(3 * variance)
        tensor.uniform_(-bound, bound)
    else:
        raise ValueError(f"invalid distribution {distribution}")


def lecun_normal_(tensor):
    variance_scaling_(tensor, mode='fan_in', distribution='truncated_normal')



def build_model_with_cfg(
        model_cls,
        variant: str,
        pretrained: bool,
        default_cfg: dict,
        model_cfg= None,
        feature_cfg= None,
        pretrained_strict: bool = True,
        pretrained_filter_fn = None,
        pretrained_custom_load = False,
        kwargs_filter = None,
        **kwargs):
    """ Build model with specified default_cfg and optional model_cfg
    This helper fn aids in the construction of a model including:
      * handling default_cfg and associated pretained weight loading
      * passing through optional model_cfg for models with config based arch spec
      * features_only model adaptation
      * pruning config / model adaptation
    Args:
        model_cls (nn.Module): model class
        variant (str): model variant name
        pretrained (bool): load pretrained weights
        default_cfg (dict): model's default pretrained/task config
        model_cfg (Optional[Dict]): model's architecture config
        feature_cfg (Optional[Dict]: feature extraction adapter config
        pretrained_strict (bool): load pretrained weights strictly
        pretrained_filter_fn (Optional[Callable]): filter callable for pretrained weights
        pretrained_custom_load (bool): use custom load fn, to load numpy or other non PyTorch weights
        kwargs_filter (Optional[Tuple]): kwargs to filter before passing to model
        **kwargs: model args passed through to model __init__
    """
    pruned = kwargs.pop('pruned', False)
    features = False
    feature_cfg = feature_cfg or {}
    default_cfg = deepcopy(default_cfg) if default_cfg else {}
    update_default_cfg_and_kwargs(default_cfg, kwargs, kwargs_filter)
    default_cfg.setdefault('architecture', variant)

    # Setup for feature extraction wrapper done at end of this fn
    if kwargs.pop('features_only', False):
        features = True
        feature_cfg.setdefault('out_indices', (0, 1, 2, 3, 4))
        if 'out_indices' in kwargs:
            feature_cfg['out_indices'] = kwargs.pop('out_indices')

    # Build the model
    model = model_cls(**kwargs) if model_cfg is None else model_cls(cfg=model_cfg, **kwargs)
    model.default_cfg = default_cfg


    # For classification models, check class attr, then kwargs, then default to 1k, otherwise 0 for feats
    num_classes_pretrained = 0 if features else getattr(model, 'num_classes', kwargs.get('num_classes', 1000))
    if pretrained:
        if pretrained_custom_load:
            load_custom_pretrained(model)
        else:
            load_pretrained(
                model,
                num_classes=num_classes_pretrained,
                in_chans=kwargs.get('in_chans', 3),
                filter_fn=pretrained_filter_fn,
                strict=pretrained_strict)
    return model

In [7]:
model=PaSST()

In [8]:
PaSST()

PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(64, 768, kernel_size=(4, 4), stride=(4, 4))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
 

In [8]:
import torch
import torch.nn as nn
import numpy as np
import math
# import cv2
import os
user_path = os.path.expanduser('~')
EPS = 1e-16


class VideoFrontend(nn.Module):
    def __init__(self):
        super(VideoFrontend, self).__init__()
        self.video_frontend = nn.Sequential(
            nn.Conv3d(1, 3, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False),
            nn.BatchNorm3d(3), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1)))
        #self.resnet = ResNet(block=BasicBlock, layers=[2, 2, 2, 2], num_classes=output_dim)
        self.passt = PaSST()
    def forward(self, x):
        B, T, _, _ = x.size()
        x = x.unsqueeze(1)
        x = self.video_frontend(x)
        x = x.transpose(1, 2).contiguous()
        x = x.view(-1, 3, x.size(3), x.size(4))
        x = self.passt(x)
        x = x[0]
        x = x.view(B, T, -1)
        return x


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(2)
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.bnfc = nn.BatchNorm1d(num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.fc(x)
        print(x.shape)
        x = self.bnfc(x)
        print(x.shape)
        return x



In [20]:
dataset_train[0][0].shape

(34, 96, 96)

In [17]:
x =dataset_train[0][0]

In [ ]:
B, T, _, _ = x.size()
        x = x.unsqueeze(1)
        x = self.video_frontend(x)
        x = x.transpose(1, 2).contiguous()
        x = x.view(-1, 64, x.size(3), x.size(4))
        x = self.passt(x)
        x = x.view(B, T, -1)

In [42]:
resnet = ResNet(block=BasicBlock, layers=[2, 2, 2, 2], num_classes=256)

In [45]:
resnet = resnet.cuda()

In [46]:
resnet(x)

torch.Size([34, 512])
torch.Size([34, 256])
torch.Size([34, 256])


tensor([[-1.6305e-01,  6.5810e-01,  2.3121e+00,  ...,  4.9449e-02,
          1.2323e+00,  1.3723e+00],
        [ 1.6004e+00,  1.2604e-02,  3.8126e-01,  ..., -1.4543e-01,
         -8.7489e-01, -8.2432e-02],
        [-2.9362e-01,  1.1488e+00,  1.0416e+00,  ..., -8.6923e-01,
          1.2141e+00, -8.8083e-01],
        ...,
        [-5.6258e-01, -1.1120e+00,  2.9112e-03,  ...,  1.1078e+00,
          2.2083e+00, -2.9774e-01],
        [ 3.4067e+00,  3.2376e+00, -1.1747e+00,  ..., -1.4241e-01,
          8.0649e-01,  1.2649e+00],
        [ 1.0837e+00,  7.9165e-02,  1.2600e+00,  ...,  1.8034e+00,
         -1.4604e+00, -2.6419e-02]], device='cuda:0',
       grad_fn=<CudnnBatchNormBackward>)

In [43]:
x.shape

torch.Size([34, 64, 24, 24])

In [46]:
ouput= resnet(x)

torch.Size([22, 512])
torch.Size([22, 1])
torch.Size([22, 1])


In [49]:
ouput.shape

torch.Size([22, 1])

NameError: name 'Passt' is not defined

In [18]:
VideoFrontend()

VideoFrontend(
  (video_frontend): Sequential(
    (0): Conv3d(1, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  )
  (resnet): ResNet(
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): Batc

In [10]:
#model parameter 확인
for name, param in nnet.named_parameters(): 
    print(f'name:{name}') 
    print(type(param)) 
    print(f'param.shape:{param.shape}') 
    print(f'param.requries_grad:{param.requires_grad}') 
    print('=====')

NameError: name 'nnet' is not defined

In [53]:
x = x.transpose(1, 2).contiguous()

In [21]:
x.shape

NameError: name 'x' is not defined

In [55]:
x = x.view(-1, 64, x.size(3), x.size(4))

In [56]:
x.shape

torch.Size([512, 64, 24, 24])

In [61]:
import math

In [67]:
B = 16

In [63]:
x = resnet(x)

In [64]:
x.shape

torch.Size([512, 256])

In [73]:
x = x.view(B, T, -1)

In [74]:
x.shape

torch.Size([16, 32, 9216])

In [75]:
b,t,96*96

9216

In [31]:
 x = x.transpose(1, 2).contiguous()

In [32]:
x = x.view(-1, 64, x.size(3), x.size(4))

RuntimeError: shape '[-1, 64, 96, 96]' is invalid for input of size 313344

In [9]:
import torch.nn

In [10]:
import torch.nn as nn
import torch
#from extract_lip_embedding_resnet import VideoFrontend

class LSTM_Encoder(nn.Module):
    def __init__(self,feature_dim,hidden_size,num_layers):
        super(LSTM_Encoder, self).__init__()
        self.feature_dim = feature_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.stack_rnn = nn.LSTM(input_size=self.feature_dim, hidden_size=self.hidden_size, batch_first=False, bidirectional=False, num_layers=1)

    def forward(self, cur_inputs, current_frame):
        packed_input = nn.utils.rnn.pack_padded_sequence(cur_inputs, current_frame)
        rnn_out, _ = self.stack_rnn(packed_input)
        rnn_out, _ = nn.utils.rnn.pad_packed_sequence(rnn_out) 
               
        return rnn_out


class KWS_Net(nn.Module):
    def __init__(self,input_dim,hidden_sizes,lstm_num_layers):
        super(KWS_Net,self).__init__()
        self.lip_encoder = VideoFrontend()
        # self.conv_av1 = nn.Conv2d(256,256, kernel_size=(1, 5), stride=(1,2), padding=(0,2))
        # self.conv_av2 = nn.Conv2d(256,512, kernel_size=(1, 5), stride=(1,2), padding=(0,2))
        self.feature_dim = input_dim
        self.hidden_size = hidden_sizes
        self.num_layers = lstm_num_layers
       
        self.encoder = LSTM_Encoder(256, self.hidden_size, self.num_layers)

        self.conv1 = nn.Conv2d(1,32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.conv2 = nn.Conv2d(32, 128, kernel_size=(5,5), stride=(2,2), padding=(2,2))
        self.max_pool1 = nn.MaxPool2d(2, stride=(1,2))
        self.conv3 = nn.Conv2d(128, 256, kernel_size=(5,5), stride=(1,2), padding=(2,2))
        self.fc1 = nn.Linear(256, 512)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
       

    def forward(self, video_inputs, current_frame):
        video_inputs = video_inputs.permute(1, 0, 2, 3)
        lip_inputs = self.lip_encoder(video_inputs) #(B, T, 256)
        lip_inputs = lip_inputs.permute(1, 0, 2) # (T, B, 256)
        
        # #lstm layer
        encoder_output = self.encoder(lip_inputs, current_frame)
        encoder_output = encoder_output.permute(1,2,0) 
        # print(encoder_output.shape) # (B,64,T)
        
        # #CNN detector and classifier 
        cnn_input = encoder_output.unsqueeze(1) 
        cnn_out = self.conv1(cnn_input)
        cnn_out = self.conv2(cnn_out)
        cnn_out = self.max_pool1(cnn_out)
        cnn_out = ((self.conv3(cnn_out)).mean(-2)).permute(0,2,1) 
        cnn_out = self.fc1(cnn_out)
        cnn_out = self.dropout(cnn_out)
        cnn_out = self.fc2(cnn_out)
        cnn_out = self.fc3(cnn_out)

        max_pool2 = nn.MaxPool2d((cnn_out.shape[1],1))
        cnn_out = max_pool2(cnn_out) 
        cnn_out = cnn_out.squeeze(-1)
 
        return cnn_out


In [11]:
nnet = KWS_Net(input_dim=256, hidden_sizes=256,lstm_num_layers=1).cuda()

In [12]:
#swa
from torchcontrib.optim import SWA
import torchcontrib

base_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, nnet.parameters()), lr=5e-5, weight_decay=1e-6)
optimizer = torchcontrib.optim.SWA(base_opt, swa_start=10, swa_freq=5, swa_lr=0.0001)


In [12]:
nnet

KWS_Net(
  (lip_encoder): VideoFrontend(
    (video_frontend): Sequential(
      (0): Conv3d(1, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    )
    (passt): PaSST(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(64, 768, kernel_size=(4, 4), stride=(4, 4))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.

In [16]:
PaSST()

PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(64, 768, kernel_size=(4, 4), stride=(4, 4))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
 

In [13]:
#optimizer = optim.Adam(nnet.parameters(), lr=5e-5)
BCE_loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(1.0)) 

In [12]:
print(nnet)

KWS_Net(
  (lip_encoder): VideoFrontend(
    (video_frontend): Sequential(
      (0): Conv3d(1, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    )
    (passt): PaSST(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(64, 768, kernel_size=(4, 4), stride=(4, 4))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.

In [68]:
# define the model
    
nnet = KWS_Net(input_dim=256, hidden_sizes=256,lstm_num_layers=1).cuda()
    #nnet = nnet.cuda()

#pretrained_dict =torch.load('/home/hj20/misp2021_baseline/task1_wws/lipreading_LRW.pt', map_location=torch.device("cuda"))
#model_dict = nnet.lip_encoder.state_dict()
#new_model_dict = dict()
#for k, v in model_dict.items():
#    if ('video_frontend.' + k) in pretrained_dict.keys() and v.size() == pretrained_dict['video_frontend.' + k].size():
#        new_model_dict[k] = pretrained_dict['video_frontend.' + k]
#new_model_dict = {k: v for k, v in new_model_dict.items()
#                if k in model_dict.keys()
#                and v.size() == model_dict[k].size()}
#nnet.lip_encoder.load_state_dict(new_model_dict)

    # training setups


In [14]:
nnet

KWS_Net(
  (lip_encoder): VideoFrontend(
    (video_frontend): Sequential(
      (0): Conv3d(1, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    )
    (passt): PaSST(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(64, 768, kernel_size=(4, 4), stride=(4, 4))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.

In [14]:
for iter_ in range(20):
    start_time = time.time()
    running_loss = 0.0
    nnet.train()
    for video_feature, data_label_torch, current_frame in dataloader_train:
        video_feature = video_feature.cuda()
    
        data_label_torch = data_label_torch.cuda()
        outputs = nnet(video_feature, current_frame)
        loss = BCE_loss(outputs, data_label_torch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    logger.info("Iteration:{0}, loss = {1:.6f} ".format(iter_, running_loss / all_file))

        # eval
    nnet.eval()
    pre_list, pre_list_d, label_list = [], [], []
    with torch.no_grad():
        running_loss_dev = 0.0
        pre_sum = 0.0
        for video_feature_dev, data_label_torch_dev, current_frame_dev in dataloader_dev:
            video_feature_dev = video_feature_dev.cuda()
            data_label_torch_dev = data_label_torch_dev.cuda()
            outputs_dev = nnet(video_feature_dev, current_frame_dev)
            loss_dev = BCE_loss(outputs_dev, data_label_torch_dev)
                
            running_loss_dev += loss_dev.item()
            outputs_dev_np = (torch.ceil(torch.sigmoid(outputs_dev)-0.5)).data.cpu().numpy()
            pre_list.extend(outputs_dev_np[:,0])
            label_list.extend(list(data_label_torch_dev.data.cpu().numpy()))
                
        TP, FP, TN, FN = utils.cal_indicator(pre_list, label_list)

        FAR = FP / (FP + TN)
        FRR = 1 - TP / (TP + FN)

        logger.info("Middle video Epoch:%d, Train loss=%.4f, Valid loss=%.4f, FAR=%.4f, FRR:%.4f" % (iter_,
                    running_loss / all_file,running_loss_dev /all_file_dev, FAR,FRR))

    torch.save(nnet.state_dict(), os.path.join(model_path, "{}_{}.model".format('kws_video', iter_)))
    nnet.train()

    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
    logger.info("*" * 50)


x torch.Size([300, 3, 24, 24])
self.norm(x) torch.Size([300, 768, 6, 6])
 patch_embed :  torch.Size([300, 768, 6, 6])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 6])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 6, 1])
X flattened torch.Size([300, 36, 768])
 self.new_pos_embed.shape torch.Size([1, 1, 768])
 self.cls_tokens.shape torch.Size([300, 1, 768])
 final sequence x torch.Size([300, 37, 768])
 after 12 atten blocks x torch.Size([300, 37, 768])
forward_features torch.Size([300, 768])
head torch.Size([300, 256])


RuntimeError: CUDA out of memory. Tried to allocate 132.00 MiB (GPU 0; 10.76 GiB total capacity; 7.93 GiB already allocated; 77.69 MiB free; 8.53 GiB reserved in total by PyTorch)

In [82]:
video_feature, data_label_torch, current_frame = next(iter(dataloader_train))
print(video_feature.shape, data_label_torch.shape, len(current_frame))
print(current_frame)

torch.Size([75, 16, 96, 96])torch.Size([75, 16, 96, 96])

torch.Size([75, 16, 96, 96])torch.Size([75, 16, 96, 96])torch.Size([75, 16, 96, 96])


torch.Size([75, 16, 96, 96])
torch.Size([75, 16, 96, 96])
torch.Size([75, 16, 96, 96])
torch.Size([75, 16, 96, 96])
torch.Size([75, 16, 96, 96])torch.Size([75, 16, 96, 96])

torch.Size([75, 16, 96, 96])
torch.Size([75, 16, 96, 96]) torch.Size([16, 1]) 16
[75, 75, 75, 75, 75, 71, 66, 50, 49, 44, 36, 34, 32, 30, 26, 25]


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--lr", default=5e-5, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=16, type=int)
parser.add_argument("--minibatchsize_dev", default=1, type=int)
parser.add_argument("--input_dim", default=256, type=int)
parser.add_argument("--hidden_sizes", default=256, type=int)
parser.add_argument("--output_dim", default=1, type=int)
parser.add_argument("--lstm_num_layers", default=1, type=int)
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--project", default='video_model', type=str)
parser.add_argument("--logdir", default='./log/', type=str)
parser.add_argument("--model_name", default='kws_video', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=20, type=int)
parser.add_argument("--gpu", default="0", type=str)
parser.add_argument("--train_num_workers", default=6, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
args = parser.parse_args()


In [16]:
!pip install vit-pytorch

     |████████████████████████████████| 50 kB 819 kB/s  eta 0:00:01


In [19]:
import torch
from vit_pytorch import ViT

v = ViT(
    image_size = 24,
    patch_size = 4,
    num_classes = 256,
    dim = 256,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

img = torch.randn(1, 64, 24, 24)

preds = v(img) # (1, 1000)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (36x1024 and 48x256)